# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding

## The event_datafile_new.csv created above contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Connection to a Cassandra instance
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

create_query1 =  """
                CREATE TABLE IF NOT EXISTS session_history
                (sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                songLength float, 
                PRIMARY KEY(sessionId, itemInSession))
                
                """
try:
    session.execute(create_query1)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'INSERT INTO session_history (sessionId, itemInSession, artist, song, songLength)'  
        query = query + 'VALUES (%s, %s, %s, %s, %s)'
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to verify that the data have been inserted into each table

In [10]:
## The table session_history was created with session_Id as the partitioning key because 
## the queries are filtering by this column. We want to know what was listend to in a particular session.
## itemInSession was used as a clustering column to make up a unique key
rows = session.execute('SELECT artist, song, songLength FROM session_history WHERE sessionId = 338 AND itemInSession = 4')

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', songlength=495.30731201171875)


### QUERY 2

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

create_query2 = """
                CREATE TABLE IF NOT EXISTS user_session_table 
                (user_id int, 
                sessionId int, 
                itemInSession int,
                artist text, 
                song text, 
                firstName text,
                lastName text,
                PRIMARY KEY((user_id, sessionId), itemInSession))
                """

try:
    session.execute(create_query2)
except Exception as e:
    print(e)



file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_table (user_id, sessionId, itemInSession, artist, song, firstName, lastName) "  
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


        
# The table user_session_table was created with user_id and sessionId as a composite partitioning key because 
## the queries are filtering by these columns. We want to know what was listend to by a user in a particular session.
## itemInSession was used as a clustering column to make up a unique key
rows = session.execute('SELECT artist, song, firstName, lastName FROM user_session_table WHERE user_id = 10 AND sessionId = 182')

for row in rows:
    print(row)    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## QUERY 3

In [12]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
                   
create_query3 = """
                CREATE TABLE IF NOT EXISTS user_songs_table
                (song text,
                user_id int, 
                firstName text,
                lastName text,
                PRIMARY KEY(song, user_id))
                """

try:
    session.execute(create_query3)
except Exception as e:
    print(e)



file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs_table (song, user_id, firstName, LastName) "  
        query = query + "VALUES (%s, %s, %s, %s) "
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


        
## The table user_songs_table was created with song as the partitioning key because 
## the queries are filtering by this column. We want to know what users listened to a particular song.
## itemInSession was used as a clustering column to make up a unique key.
rows = session.execute("""
                        SELECT firstName, lastName
                        FROM user_songs_table 
                        WHERE song = 'All Hands Against His Own'
                        """)

for row in rows:
    print(row)    


Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [13]:
try:
    session.execute('DROP TABLE session_history')
except Exception as e:
    print(e)
    
    
try:
    session.execute('DROP TABLE user_session_table')
except Exception as e:
    print(e)    

    
try:
    session.execute('DROP TABLE user_songs_table')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()